In [1]:
import os
import math
import pandas as pd
import datetime as dt
import calendar
import numpy as np

In [2]:
dlst = [dt.datetime(2019,3,31,3,30), dt.datetime(2020,3,29,3,30), dt.datetime(2021,3,28,3,30), dt.datetime(2022,3,27,3,30)]
march_dlst_days = {2019: 31, 2020: 29, 2021: 28, 2022: 27, 2023: 26}

In [3]:
market_prices_dir = 'data/market/dam_idm_prices/'
unbalance_prices_dir = 'data/market/unbalance_prices/'


In [4]:
dam_idm_prices_files =  [file for file in os.listdir(market_prices_dir) if os.path.isfile(os.path.join(market_prices_dir, file))]
# dam_idm_prices_files.remove('desktop.ini')

print(len(dam_idm_prices_files))
print(dam_idm_prices_files)

2
['price_DAM_IDM_01.2022.xls', 'price_DAM_IDM_02.2022.xls']


### Reading DAM prices

In [5]:
file = 'price_DAM_IDM_02.2022.xls'
print(file)

sheet_name = 'Ціна_РДН_ОЕС_України'

date = file[-11:-4]
date = dt.datetime.strptime(date, '%m.%Y')

data = pd.read_excel(market_prices_dir + file, sheet_name=sheet_name, header=None)
data = data.drop(index=[0, 1, 2]).reset_index().drop(columns=['index', 0])
squized = data.stack()
print(squized)

squized = squized.reset_index(level=[0,1])
squized['level_0'] = squized['level_0'] + 1

squized.columns = ['day', 'hour','dam_price']    
squized = squized.astype({'day': 'int', 'hour': 'int', 'dam_price': 'float'})

if date.month == 3:
    #print('MONTH is MARCH!!!!')
    dlst_day = march_dlst_days[date.year]
    squized = squized.drop(squized[(squized.day == dlst_day) & (squized.hour == 24)].index)

start = dt.datetime(year=date.year, month=date.month, day=1, hour=0, minute=30)

last_month_day_calendar = calendar.monthrange(date.year, date.month)[-1]
last_month_day_data = squized['day'].max()

if last_month_day_data == last_month_day_calendar:
    year = date.year
    month = date.month + 1 
    if month == 13:
        month = 1
        year = year + 1
    end = dt.datetime(year=year, month=month, day=1, hour=0, minute=30) - dt.timedelta(hours=1)
else:
    end = dt.datetime(year=date.year, month=date.month, day=last_month_day_data + 1, hour=0, minute=30)
    end = end - dt.timedelta(hours=1)

#print(last_month_day_calendar, last_month_day_data)
# print(start, end)

index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)

#print(squized.tail(52))

squized.index = index_in_utc
squized = squized.drop(columns=['day', 'hour'])

dam_prices = squized

# print(data.head(10))
print(date)
print('ok!')

print(dam_prices)


price_DAM_IDM_02.2022.xls
0   1       1440
    2     1400.0
    3     1000.0
    4     1000.0
    5     1000.0
           ...  
13  20    2550.0
    21    2550.0
    22    2550.0
    23    2550.0
    24     987.5
Length: 336, dtype: object
2022-02-01 00:00:00
ok!
                     dam_price
2022-01-31 22:30:00     1440.0
2022-01-31 23:30:00     1400.0
2022-02-01 00:30:00     1000.0
2022-02-01 01:30:00     1000.0
2022-02-01 02:30:00     1000.0
...                        ...
2022-02-14 17:30:00     2550.0
2022-02-14 18:30:00     2550.0
2022-02-14 19:30:00     2550.0
2022-02-14 20:30:00     2550.0
2022-02-14 21:30:00      987.5

[336 rows x 1 columns]


### Reading IDM prices

In [6]:
file = file
print(file)

sheet_name = 'Ціна_ВДР_ОЕС_України'

idm_prices = dict()

date = file[-11:-4]
date = dt.datetime.strptime(date, '%m.%Y')

data = pd.read_excel(market_prices_dir + file, sheet_name=sheet_name, header=None)
data = data.drop(index=[0, 1, 2]).reset_index().drop(columns=['index', 0])

#data['datetime'] = dt.datetime(year=date.year, month=date.month, day=data[0])
squized = data.stack()

squized = squized.reset_index(level=[0,1])
squized['level_0'] = squized['level_0'] + 1

squized.columns = ['day', 'hour','idm_price']    
squized = squized.astype({'day': 'int', 'hour': 'int', 'idm_price': 'float'})

if date.month == 3:
    #print('MONTH is MARCH!!!!')
    dlst_day = march_dlst_days[date.year]
    squized = squized.drop(squized[(squized.day == dlst_day) & (squized.hour == 24)].index)

start = dt.datetime(year=date.year, month=date.month, day=1, hour=0, minute=30)

last_month_day_calendar = calendar.monthrange(date.year, date.month)[-1]
last_month_day_data = squized['day'].max()

if last_month_day_data == last_month_day_calendar:
    year = date.year
    month = date.month + 1 
    if month == 13:
        month = 1
        year = year + 1
    end = dt.datetime(year=year, month=month, day=1, hour=0, minute=30) - dt.timedelta(hours=1)
else:
    end = dt.datetime(year=date.year, month=date.month, day=last_month_day_data + 1, hour=0, minute=30)
    end = end - dt.timedelta(hours=1)

index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)

#print(squized.tail(52))

squized.index = index_in_utc
squized = squized.drop(columns=['day', 'hour'])

idm_prices = squized
#print(data.head(10))
print(date)
print('ok!')

print(idm_prices)

price_DAM_IDM_02.2022.xls
2022-02-01 00:00:00
ok!
                     idm_price
2022-01-31 22:30:00    1406.39
2022-01-31 23:30:00    1372.56
2022-02-01 00:30:00     974.90
2022-02-01 01:30:00     973.57
2022-02-01 02:30:00     974.00
...                        ...
2022-02-13 17:30:00    4000.00
2022-02-13 18:30:00    4000.00
2022-02-13 19:30:00    4000.00
2022-02-13 20:30:00    3982.91
2022-02-13 21:30:00    1481.25

[312 rows x 1 columns]


In [8]:
ub_prices_files =  [file for file in os.listdir(unbalance_prices_dir) if os.path.isfile(os.path.join(unbalance_prices_dir, file))]
# ub_prices_files.remove('desktop.ini')

print(ub_prices_files)

['Faktychni-tsiny-nebalansiv-01-09.02.2022.xlsx', 'Faktychni-tsiny-nebalansiv-01-13.02.2022.xlsx', 'Faktychni-tsiny-nebalansiv-01-18.01.2022.xlsx', 'Faktychni-tsiny-nebalansiv-01-20.01.2022.xlsx', 'Faktychni-tsiny-nebalansiv-01-26.01.2022.xlsx', 'Faktychni-tsiny-nebalansiv-01-31.01.2022.xlsx']


### Reading IMSP prices

In [9]:
file = 'Faktychni-tsiny-nebalansiv-01-13.02.2022.xlsx'
print(file)

sheet_name = 'Аркуш1'

ub_prices = dict()
    
date = file[-12:-5]
date = dt.datetime.strptime(date, '%m.%Y')

data = pd.read_excel(unbalance_prices_dir + file, index_col=[0, 1], usecols=[0, 1, 2], header=0, skiprows=[1, 2], parse_dates=[0])
data.columns = ['imsp']
data = data.astype({'imsp': 'float'})
time_deltas = [dt.timedelta(hours=int(record[-1][:2]), minutes=30) for record in data.index.values]
data.index = [record[0] + dt.timedelta(hours=int(record[-1][:2]), minutes=30) for record in data.index.values]
data.dropna(inplace=True)

if date.month == 3:
    print('MONTH is MARCH!!!!')
    dlst_day = march_dlst_days[date.year]
    print(dlst_day)
    #print(data)
    #print(data[(data.index.day == dlst_day) & (data.index.hour == 23)].index)
    #data = data.drop(data[(data.index.day == dlst_day) & (data.index.hour == 23)].index)
    #print(data)

#print(squized.tail(52))


start = dt.datetime(year=date.year, month=date.month, day=1, hour=0, minute=30)

last_month_day_calendar = calendar.monthrange(date.year, date.month)[-1]
last_month_day_data = data.index.day.max()

if last_month_day_data == last_month_day_calendar:
    year = date.year
    month = date.month + 1 
    if month == 13:
        month = 1
        year = year + 1
    end = dt.datetime(year=year, month=month, day=1, hour=0, minute=30) - dt.timedelta(hours=1)
else:
    end = dt.datetime(year=date.year, month=date.month, day=last_month_day_data, hour=23, minute=30)

#print(last_month_day_calendar, last_month_day_data)
#print(start, end)

index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)

#print(squized.tail(52))

data.index = index_in_utc

ub_prices = data

print(date)
print('Ok!')

print(ub_prices)


Faktychni-tsiny-nebalansiv-01-13.02.2022.xlsx
2022-02-01 00:00:00
Ok!
                        imsp
2022-01-31 22:30:00     0.01
2022-01-31 23:30:00     0.01
2022-02-01 00:30:00     0.01
2022-02-01 01:30:00     0.01
2022-02-01 02:30:00     0.01
...                      ...
2022-02-13 17:30:00  4600.00
2022-02-13 18:30:00  4600.00
2022-02-13 19:30:00  4600.00
2022-02-13 20:30:00  4599.99
2022-02-13 21:30:00  2299.99

[312 rows x 1 columns]


### Preparing intersected data

In [10]:
print(dam_prices.shape)
print(idm_prices.shape)
print(ub_prices.shape)

(336, 1)
(312, 1)
(312, 1)


In [11]:
print(dam_prices.loc[dam_prices.index.duplicated(keep='first')])
print(idm_prices.loc[idm_prices.index.duplicated(keep='first')])
print(ub_prices.loc[ub_prices.index.duplicated(keep='first')])

Empty DataFrame
Columns: [dam_price]
Index: []
Empty DataFrame
Columns: [idm_price]
Index: []
Empty DataFrame
Columns: [imsp]
Index: []


In [12]:
prices = pd.concat([dam_prices, idm_prices, ub_prices], join='outer', axis=1)
prices = prices.dropna()
prices.index.name = 'datetime'

prices['positive_ub_price'] = (prices[['dam_price', 'imsp']].min(axis=1) * (1 - 0.05)).apply(lambda x: round(x, 2))
prices['negative_ub_price'] = (prices[['dam_price', 'imsp']].max(axis=1) * (1 + 0.05)).apply(lambda x: round(x, 2))
print(prices)
#prices['idm_price'] = idm_prices_1
#prices['imsp_price'] = ub_prices_1

#prices = dam_prices_1.merge(idm_prices_1, left_index=True, right_index=True).merge(ub_prices_1, left_index=True, right_index=True)
# prices.to_excel('prices.xlsx')
print('Ok!')
print(prices.info())
print(prices)

                     dam_price  idm_price     imsp  positive_ub_price  \
datetime                                                                
2022-01-31 22:30:00    1440.00    1406.39     0.01               0.01   
2022-01-31 23:30:00    1400.00    1372.56     0.01               0.01   
2022-02-01 00:30:00    1000.00     974.90     0.01               0.01   
2022-02-01 01:30:00    1000.00     973.57     0.01               0.01   
2022-02-01 02:30:00    1000.00     974.00     0.01               0.01   
...                        ...        ...      ...                ...   
2022-02-13 17:30:00    2750.00    4000.00  4600.00            2612.50   
2022-02-13 18:30:00    2750.00    4000.00  4600.00            2612.50   
2022-02-13 19:30:00    2716.13    4000.00  4600.00            2580.32   
2022-02-13 20:30:00    2655.27    3982.91  4599.99            2522.51   
2022-02-13 21:30:00     987.50    1481.25  2299.99             938.12   

                     negative_ub_price  
datetime 

### Putting data into database

In [13]:
from sqlalchemy import create_engine, MetaData, desc
from sqlalchemy.sql import select, insert, update, and_, or_, not_, func
from sqlalchemy.pool import NullPool
from settings.db import DO_URL


engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

with engine.connect() as connection:

    emp_table = metadata.tables['electricity_market_prices']
    query_1 = select([emp_table.c.id, emp_table.c.completed]).where(and_(emp_table.c.year == date.year, emp_table.c.month == date.month))
    complete_period = prices.index.max().day == calendar.monthrange(date.year, date.month)[1]
    record_to_update = connection.execute(query_1).fetchall()
    print(record_to_update)
    if record_to_update:
        if not record_to_update[0][1]:
            update_statement = emp_table.update().values((record_to_update[0][0], date.year, date.month, 
                                                      list(prices.index), 
                                                      list(prices.dam_price), 
                                                      list(prices.idm_price), 
                                                      list(prices.imsp),
                                                      list(prices.positive_ub_price),
                                                      list(prices.negative_ub_price),
                                                      complete_period)).where(emp_table.c.id == record_to_update[0][0])
            updated_id = connection.execute(update_statement)
            print('Database record updated!')
        else:
            print('Not updated! Record already completed!')
    else:
        query_2 = select([func.max(emp_table.c.id)])
        max_id = connection.execute(query_2).fetchall()
        print(max_id)
        max_id = max_id[0][0] + 1 if max_id else 1
        print(max_id)

        insert_statement = emp_table.insert().values((max_id, date.year, date.month, 
                                                      list(prices.index), 
                                                      list(prices.dam_price), 
                                                      list(prices.idm_price), 
                                                      list(prices.imsp),
                                                      list(prices.positive_ub_price),
                                                      list(prices.negative_ub_price),
                                                      complete_period))
        inserted_id = connection.execute(insert_statement)
        print('Data inserted to database at index {}'.format(inserted_id))

C:\Users\n.kozelets\AppData\Local\Temp\ipykernel_13468\1650978460.py:9: SAWarning: Did not recognize type 'point' of column 'location'
  metadata.reflect(bind=engine)


[(14, False)]
Database record updated!
